In [ ]:
import warnings
warnings.filterwarnings('ignore')
import ipywidgets as widgets
output = widgets.Output()
from utils import ContentManager, DisplayManager, getWindowWidth

In [ ]:
content_manager = ContentManager(getWindowWidth())
display_manager = DisplayManager(output, content_manager)
content_manager.plotXDataDashboard()
content_manager.plotYDataDashboard()

# 模型預測

In [ ]:
display_manager.displayHyperParamDashboard()
content_manager.plotPredictionData()

# 使用自己的資料

In [ ]:
display_manager.displayValidDashboard()